In [ ]:
# 데이터 처리 및 분석
import numpy as np
import pandas as pd
import warnings
import sys
from pathlib import Path
from geopy.geocoders import Nominatim

# 시각화
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
import folium
import statsmodels.api as sm

# 통계 분석
from scipy import stats
from scipy.stats import shapiro, levene, ttest_ind, f_oneway
from scipy.stats import mannwhitneyu, kruskal, chi2, chisquare, chi2_contingency, fisher_exact
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison
import pingouin as pg
import scikit_posthocs as sp

# 로컬 모듈
sys.path.append(str(Path.cwd().parent))
sys.path.append(str(Path.cwd().parent / 'src'))
from src.config import DATA_DIR, FONT_DIR

# 출력 설정
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

import platform

# 한글 폰트 설정
if platform.system() == "Windows":
    plt.rcParams["font.family"] = "Malgun Gothic"
elif platform.system() == "Darwin":  # macOS
    plt.rcParams["font.family"] = "AppleGothic"
else:  # Linux
    plt.rcParams["font.family"] = "NanumGothic"

plt.rcParams["axes.unicode_minus"] = False
plt.rcParams["figure.figsize"] = (12, 6)

mpl.rcParams['axes.unicode_minus'] = False
# 폰트 개인 경로에 맞춰서 변경
font_path = FONT_DIR / 'FREESENTATION-6SEMIBOLD.ttf'
prop = fm.FontProperties(fname=font_path)

print("="*60)
print("라이브러리 로드 완료!")
print("한글 폰트 설정 완료!")
print("="*60)

라이브러리 로드 완료!
한글 폰트 설정 완료!


In [2]:
# 데이터셋 불러오기 1
RAW_DIR = DATA_DIR / 'raw'
customers_df = pd.read_csv(RAW_DIR / "olist_customers_dataset.csv")
geolocation_df = pd.read_csv(RAW_DIR / "olist_geolocation_dataset.csv")
order_items_df = pd.read_csv(RAW_DIR / "olist_order_items_dataset.csv")
order_payments_df = pd.read_csv(RAW_DIR / "olist_order_payments_dataset.csv")
order_reviews_df = pd.read_csv(RAW_DIR / "olist_order_reviews_dataset.csv")
orders_df = pd.read_csv(RAW_DIR / "olist_orders_dataset.csv")
products_df = pd.read_csv(RAW_DIR / "olist_products_dataset.csv")
sellers_df = pd.read_csv(RAW_DIR / "olist_sellers_dataset.csv")
category_df = pd.read_csv(RAW_DIR / "product_category_name_translation.csv")

# 00 컬럼명 변경 및 타입 변환

In [3]:
CATEGORY_MAPPING = {
    # 패션·의류
    "fashion_calcados": "패션·의류",
    "fashion_bolsas_e_acessorios": "패션·의류",
    "fashion_underwear_e_moda_praia": "패션·의류",
    "fashion_roupa_masculina": "패션·의류",
    "fashion_roupa_feminina": "패션·의류",
    "fashion_roupa_infanto_juvenil": "패션·의류",
    "fashion_esporte": "패션·의류",
    "malas_acessorios": "패션·의류",
    "relogios_presentes": "패션·의류",
    
    # 가구·인테리어
    "moveis_decoracao": "가구·인테리어",
    "moveis_escritorio": "가구·인테리어",
    "moveis_sala": "가구·인테리어",
    "moveis_cozinha_area_de_servico_jantar_e_jardim": "가구·인테리어",
    "moveis_quarto": "가구·인테리어",
    "moveis_colchao_e_estofado": "가구·인테리어",
    "cama_mesa_banho": "가구·인테리어",
    "casa_conforto": "가구·인테리어",
    "casa_conforto_2": "가구·인테리어",
    "flores": "가구·인테리어",
    "artigos_de_festas": "가구·인테리어",
    "artigos_de_natal": "가구·인테리어",
    
    # 가전제품
    "eletrodomesticos": "가전제품",
    "eletrodomesticos_2": "가전제품",
    "eletroportateis": "가전제품",
    "climatizacao": "가전제품",
    "portateis_casa_forno_e_cafe": "가전제품",
    "portateis_cozinha_e_preparadores_de_alimentos": "가전제품",
    
    # 생활용품
    "utilidades_domesticas": "생활용품",
    "la_cuisine": "생활용품",
    
    # 건축·공구
    "casa_construcao": "건축·공구",
    "construcao_ferramentas_seguranca": "건축·공구",
    "construcao_ferramentas_construcao": "건축·공구",
    "construcao_ferramentas_ferramentas": "건축·공구",
    "construcao_ferramentas_iluminacao": "건축·공구",
    "construcao_ferramentas_jardim": "건축·공구",
    "ferramentas_jardim": "건축·공구",
    
    # 전자·IT
    "eletronicos": "전자·IT",
    "informatica_acessorios": "전자·IT",
    "pcs": "전자·IT",
    "pc_gamer": "전자·IT",
    "telefonia": "전자·IT",
    "telefonia_fixa": "전자·IT",
    "tablets_impressao_imagem": "전자·IT",
    "consoles_games": "전자·IT",
    "audio": "전자·IT",
    "cine_foto": "전자·IT",
    "sinalizacao_e_seguranca": "전자·IT",
    
    # 건강·뷰티
    "beleza_saude": "건강·뷰티",
    "perfumaria": "건강·뷰티",
    "fraldas_higiene": "건강·뷰티",
    
    # 스포츠·레저
    "esporte_lazer": "스포츠·레저",
    "brinquedos": "스포츠·레저",
    "instrumentos_musicais": "스포츠·레저",
    "artes": "스포츠·레저",
    "artes_e_artesanato": "스포츠·레저",
    "cool_stuff": "스포츠·레저",
    "papelaria": "스포츠·레저",
    
    # 유아·아동
    "bebes": "유아·아동",
    
    # 자동차
    "automotivo": "자동차",
    
    # 식품·음료
    "alimentos": "식품·음료",
    "alimentos_bebidas": "식품·음료",
    "bebidas": "식품·음료",
    
    # 도서·미디어
    "livros_interesse_geral": "도서·미디어",
    "livros_tecnicos": "도서·미디어",
    "livros_importados": "도서·미디어",
    "musica": "도서·미디어",
    "cds_dvds_musicais": "도서·미디어",
    "dvds_blu_ray": "도서·미디어",
    
    # 기타
    "pet_shop": "기타",
    "agro_industria_e_comercio": "기타",
    "industria_comercio_e_negocios": "기타",
    "market_place": "기타",
    "seguros_e_servicos": "기타",
}

In [4]:
display(order_reviews_df[order_reviews_df['order_id'] == 'f014aa85d8f495d824d120e9035faacb'])

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
9126,eb8d43b4f99e45d3c7a08abd8ef1ff41,f014aa85d8f495d824d120e9035faacb,5,NaN,NaN,2018-04-12 00:00:00,2018-04-13 11:41:43
25486,86788dd2f0472ea4fbbcbe0daf10e267,f014aa85d8f495d824d120e9035faacb,4,NaN,NaN,2018-04-12 00:00:00,2018-04-14 20:59:33


In [5]:
# 불필요한 컬럼 삭제
customers_df["customer_zip_code_prefix"] = customers_df["customer_zip_code_prefix"].astype(str)
customers_df.drop(["customer_city", "customer_state"], axis=1, inplace=True)

# 컬럼명 변경 및 타입 변환
geolocation_df = geolocation_df.rename(columns={
    "geolocation_zip_code_prefix" : "zip_code", 
    "geolocation_lng" : "longitude", 
    "geolocation_lat":"latitude", 
    "geolocation_city" : "city", 
    "geolocation_state" : "state"}, 
    inplace=False)
geolocation_df["zip_code"] = geolocation_df["zip_code"].astype(int)

#우편번호 기준으로 중복된 위치는 최빈값으로 통일 
geo_mode_df = geolocation_df.groupby("zip_code").agg({
    "latitude": lambda x: x.mode().iloc[0],
    "longitude": lambda x: x.mode().iloc[0],
    "city": lambda x: x.mode().iloc[0],
    "state": lambda x: x.mode().iloc[0]}).reset_index()

#컬럼명 변경 및 타입 변환
order_items_df.rename(columns={
    "order_item_id" : "quantity",
    "shipping_limit_date": "carrier_delivered_limit_time",}, inplace=True)

# 컬럼명 변경 및 타입 변환
order_payments_df.rename(columns={"payment_value": "payment_amount"}, inplace=True)

# 한 주문에 여러 리뷰가 달린 경우 생성일이 같으면 제거, 다르면 최신만 살림
order_reviews_df.drop_duplicates(['order_id', 'review_creation_date'], keep=False, inplace=True)
order_reviews_df.sort_values('review_creation_date', inplace=True)
order_reviews_df.drop_duplicates(['order_id'], keep='last', inplace=True)
# 불필요한 컬럼 제거 및 타입 변환
order_reviews_df.drop(["review_id", "review_comment_title", "review_comment_message", "review_creation_date", "review_answer_timestamp"], axis=1, inplace=True)

# "delivered" 상태인 주문만 필터링
orders_df = orders_df[orders_df["order_status"] == "delivered"]

# 컬럼명 변경
orders_df.rename(columns={
    "order_purchase_timestamp": "purchase_time",
    "order_approved_at": "approved_time",
    "order_delivered_carrier_date": "carrier_delivered_time",
    "order_delivered_customer_date": "delivered_time",
    "order_estimated_delivery_date": "estimated_delivery_time"}, inplace=True)

# 컬럼명 변경 및 불필요한 컬럼 제거
products_df.rename(columns={
    "product_description_lenght": "content_length", 
    "product_photos_qty" : "photos_qty", 
    "product_weight_g" : "weight_g",
    "product_length_cm" : "length",
    "product_height_cm" : "height",
    "product_width_cm" : "width"
    }, inplace=True)
products_df.drop(columns=["product_name_lenght"], axis=1, inplace=True)

# 결측치를 0으로 채우고 타입 변환
products_df.fillna(0, inplace=True)

# 불필요한 컬럼 제거 및 타입 변환
sellers_df["seller_zip_code_prefix"] = sellers_df["seller_zip_code_prefix"].astype(str)
sellers_df.drop(["seller_city", "seller_state"], axis=1, inplace=True)

# 카테고리 컬럼 이름 맵핑
category_df["category"] = category_df["product_category_name"].map(CATEGORY_MAPPING)
category_df["category"] = category_df["category"].fillna("기타")

# 01 geolocation 이상치 처리

In [6]:
# 하버사인 거리 계산 함수
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # 지구 반지름 (km)
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    d_phi = np.radians(lat2 - lat1)
    d_lambda = np.radians(lon2 - lon1)
    
    a = np.sin(d_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(d_lambda / 2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    return R * c

# 기준 좌표 (첫 번째 행 기준)
base_lat = geo_mode_df.loc[0, "latitude"]
base_lon = geo_mode_df.loc[0, "longitude"]

# 각 행마다 기준점과 거리 계산해 distance_km 컬럼 추가
geo_mode_df["distance_km"] = geo_mode_df.apply(
    lambda row: haversine(base_lat, base_lon, row["latitude"], row["longitude"]),
    axis=1
)

# 4000km 이상 떨어진 데이터 필터
far_5000_df = geo_mode_df[geo_mode_df["distance_km"] >= 4000][["zip_code", "latitude", "longitude", "distance_km"]]

# 4000km 이상 떨어진 데이터 필터
for i in far_5000_df["zip_code"]:
    print(geolocation_df[geolocation_df["zip_code"] == i])
    print(f" >>> {i} => 우편번호 개수 : {geolocation_df[geolocation_df['zip_code'] == i]['zip_code'].count()}")

# 지도의 초기 중심 좌표 (첫 번째 지점을 기준)
marker = folium.Map(location=[far_5000_df["latitude"].iloc[0], far_5000_df["longitude"].iloc[0]], zoom_start=5)

# 각 지점에 마커 추가
for idx, row in far_5000_df.iterrows():
    folium.Marker(location=[row["latitude"], row["longitude"]]).add_to(marker)
    
# 우편번호 입력시 위도, 경도 찾아주는 함수
def get_lat_lon_from_zip(zip_code, country_code="BR", country_name="Brazil"):
    geolocator = Nominatim(user_agent="my-geocoder-app")
    try:
        # country_codes로 검색 범위 제한
        location = geolocator.geocode(
            f"{zip_code}, {country_name}",
            country_codes=country_code.lower(),  # "br"
            addressdetails=True
        )
        if location:
            return (location.latitude, location.longitude)
        else:
            return (None, None)
    except Exception as e:
        print(f"Error geocoding ZIP code {zip_code}: {e}")
        return (None, None)

# Example usage
zip_code = "95130-000"
latitude, longitude = get_lat_lon_from_zip(zip_code, "BR")

if latitude and longitude:
    print(f"ZIP code {zip_code}: {latitude}, {longitude}")
else:
    print(f"Could not find coordinates for ZIP code {zip_code}")

# 4000km 이상 떨어진 데이터 필터
far_5000_df["zip_code"] = far_5000_df["zip_code"].astype(str)
for i in far_5000_df["zip_code"]:
    zip_code = i + "-000"
    latitude, longitude = get_lat_lon_from_zip(zip_code, "BR")
    
    if latitude and longitude:
        print(f"ZIP code {zip_code}: {latitude}, {longitude}")
    else:
        print(f"Could not find coordinates for ZIP code {zip_code}")

result_list = []
far_5000_df["zip_code"] = far_5000_df["zip_code"].astype(str)

for i in far_5000_df["zip_code"]:
    zip_code_full = i + "-000"
    
    latitude, longitude = get_lat_lon_from_zip(zip_code_full, "BR")
    
    if latitude is not None and longitude is not None:
        result_list.append({
            "zip_code": zip_code_full,
            "latitude": latitude,
            "longitude": longitude
        })
    else:
        print(f"Could not find coordinates for ZIP code {zip_code_full}")

good_zipcode_df = pd.DataFrame(result_list)
good_zipcode_df["real_zip_code"] = good_zipcode_df["zip_code"].str[:5]

geo_mode_df["zip_code"] = geo_mode_df["zip_code"].astype(str)
zipcode_merged_df = pd.merge(geo_mode_df, good_zipcode_df, left_on="zip_code", right_on="real_zip_code", how="left", suffixes=("", "_good"))

zipcode_merged_df["latitude"] = zipcode_merged_df["latitude_good"].combine_first(zipcode_merged_df["latitude"])
zipcode_merged_df["longitude"] = zipcode_merged_df["longitude_good"].combine_first(zipcode_merged_df["longitude"])

# 불필요한 컬럼 삭제
zipcode_merged_df = zipcode_merged_df.drop(columns=["distance_km", "zip_code_good", "real_zip_code", "latitude_good", "longitude_good"], axis=1)

geo_df = zipcode_merged_df
geo_df

        zip_code   latitude  longitude                     city state
387565     18243  28.008978 -15.536867  bom retiro da esperanca    SP
 >>> 18243 => 우편번호 개수 : 1
        zip_code   latitude   longitude                    city state
538385     29654 -19.823680  -40.655474  santo antonio do canaa    ES
538512     29654  29.409252  -98.484121  santo antônio do canaã    ES
538557     29654  21.657547 -101.466766  santo antonio do canaa    ES
538584     29654 -19.825000  -40.654440  santo antonio do canaa    ES
 >>> 29654 => 우편번호 개수 : 4
        zip_code   latitude  longitude             city state
769333     68275  41.146203  -8.577855  porto trombetas    PA
769351     68275  42.166805  -6.898531  porto trombetas    PA
769356     68275  -1.743457 -52.244269  porto trombetas    PA
769368     68275  -1.743515 -52.244163  porto trombetas    PA
769391     68275  42.167251  -6.898559  porto trombetas    PA
769414     68275  -1.743457 -52.244269  porto trombetas    PA
769436     68275  42.167

,zip_code,latitude,longitude,city,state
0,1001,-23.550498,-46.634338,sao paulo,SP
1,1002,-23.548551,-46.635072,sao paulo,SP
2,1003,-23.549044,-46.635183,sao paulo,SP
3,1004,-23.549535,-46.634307,sao paulo,SP
4,1005,-23.548758,-46.638411,sao paulo,SP
...,...,...,...,...,...
19010,99960,-27.954114,-52.031943,charrua,RS
19011,99965,-28.210845,-52.059927,agua santa,RS
19012,99970,-28.343273,-51.873734,ciriaco,RS
19013,99980,-28.388342,-51.845194,david canabarro,RS


## 02 승인 이상치 해결

In [7]:
time_cond = orders_df['purchase_time'] <= orders_df['approved_time'] 
time_cond &= (orders_df['approved_time'] <= orders_df['carrier_delivered_time'] )
time_cond &= orders_df['carrier_delivered_time'] <= orders_df['delivered_time']
orders_df[~time_cond]

,order_id,customer_id,order_status,purchase_time,approved_time,carrier_delivered_time,delivered_time,estimated_delivery_time
15,dcb36b511fcac050b97cd5c05de84dc3,3b6828a50ffe546942b7a473d70ac0fc,delivered,2018-06-07 19:03:12,2018-06-12 23:31:02,2018-06-11 14:54:00,2018-06-21 15:34:32,2018-07-04 00:00:00
64,688052146432ef8253587b930b01a06d,81e08b08e5ed4472008030d70327c71f,delivered,2018-04-22 08:48:13,2018-04-24 18:25:22,2018-04-23 19:19:14,2018-04-24 19:31:58,2018-05-15 00:00:00
199,58d4c4747ee059eeeb865b349b41f53a,1755fad7863475346bc6c3773fe055d3,delivered,2018-07-21 12:49:32,2018-07-26 23:31:53,2018-07-24 12:57:00,2018-07-25 23:58:19,2018-07-31 00:00:00
210,412fccb2b44a99b36714bca3fef8ad7b,c6865c523687cb3f235aa599afef1710,delivered,2018-07-22 22:30:05,2018-07-23 12:31:53,2018-07-23 12:24:00,2018-07-24 19:26:42,2018-07-31 00:00:00
415,56a4ac10a4a8f2ba7693523bb439eede,78438ba6ace7d2cb023dbbc81b083562,delivered,2018-07-22 13:04:47,2018-07-27 23:31:09,2018-07-24 14:03:00,2018-07-28 00:05:39,2018-08-06 00:00:00
...,...,...,...,...,...,...,...,...
99091,240ead1a7284667e0ec71d01f80e4d5e,fcdd7556401aaa1c980f8b67a69f95dc,delivered,2018-07-02 16:30:02,2018-07-05 16:17:59,2018-07-05 14:11:00,2018-07-10 23:21:47,2018-07-24 00:00:00
99230,78008d03bd8ef7fcf1568728b316553c,043e3254e68daf7256bda1c9c03c2286,delivered,2018-07-03 13:11:13,2018-07-05 16:32:52,2018-07-03 12:57:00,2018-07-10 17:47:39,2018-07-23 00:00:00
99266,76a948cd55bf22799753720d4545dd2d,3f20a07b28aa252d0502fe7f7eb030a9,delivered,2018-01-30 02:41:30,2018-02-04 23:31:46,2018-01-31 18:11:58,2018-03-18 20:08:50,2018-03-02 00:00:00
99377,a6bd1f93b7ff72cc348ca07f38ec4bee,6d63fa86bd2f62908ad328325799152f,delivered,2018-04-20 17:28:40,2018-04-24 19:26:10,2018-04-23 17:18:40,2018-04-28 17:38:42,2018-05-15 00:00:00


In [8]:
orders_df = orders_df.loc[time_cond]

# 03 구매자 데이터프레임 생성

In [9]:
### 1 customers_df + geo_mode_df
# 구매자 우편번호에서 위치정보 데이터프레임에도 존재하지 않는 우편번호 278개 삭제
# 중복되는 필요없는 컬럼들 삭제
merged_customers_df = pd.merge(customers_df, geo_df, left_on="customer_zip_code_prefix", right_on="zip_code", how="left")
merged_customers_df.dropna(subset=["zip_code"], inplace=True)
merged_customers_df.drop(["customer_zip_code_prefix"], axis=1, inplace=True)


### 2 + orders_df           (99163, 14) -> (96191, 14)
# 주문완료된 건만 
# 주문 승인 날짜, 배송 파트너에게 주문 인계한 날짜, 실제로 배송 완료된 날짜 컬럼에 결측치 삭제
customer_orders_df = pd.merge(merged_customers_df, orders_df, on="customer_id", how="left")
customer_orders_df = customer_orders_df[customer_orders_df["order_status"] == "delivered"] 
customer_orders_df.dropna(subset=["approved_time", "carrier_delivered_time", "delivered_time"], inplace=True)
    # 필요없는 컬럼 삭제
# customer_orders_df.drop(["order_status"], axis=1, inplace=True)


### 3 + order_payments_df           (100462, 18) -> (100459, 18)
# 결측치 삭제
# 일시불은 1값인데, 0의 값 2개는 결측치로 판단하여 제거
customer_orders_payments_df = pd.merge(customer_orders_df, order_payments_df, on="order_id", how="left")
customer_orders_payments_df.dropna(subset=["payment_sequential"], inplace=True)
customer_orders_payments_df.drop(customer_orders_payments_df[customer_orders_payments_df["payment_installments"] == 0].index, inplace=True)


### 4 + order_items_df             (114710, 24) -> (114710, 23)
# order_items에 없는 주문 제거
# 필요없는 컬럼 삭제
customer_items_df = pd.merge(customer_orders_payments_df, order_items_df, on="order_id", how="left")
customer_items_df.dropna(subset=["product_id"], inplace=True)
customer_items_df.drop(["seller_id"], axis=1, inplace=True)

### 5 + order_reviews_df               (115390, 24) -> (115390, 24)
# 리뷰 점수 결측치 "리뷰없음" 으로 대체
customer_review_df = pd.merge(customer_items_df, order_reviews_df, on="order_id", how="left")
customer_review_df["review_score"].fillna("리뷰없음", inplace=True)


### 6 + products_df                    (115390, 28)  ->  (115390, 28)
# products에 없는 상품 제거
customer_products_df = pd.merge(customer_review_df, products_df, on="product_id", how="left")
customer_products_df.dropna(subset=["product_category_name"], inplace=True)


### 7 + category_df               (115390, 30)  ->   (115390, 30)
# 카테고리 없는 결측치 대체
customer_category_df = pd.merge(customer_products_df, category_df, on="product_category_name", how="left")
customer_category_df["category"].fillna("카테고리없음", inplace=True)


### 8 필터링              (115390, 30)
# 2017-01-01 이후 거래건 + 2018-09-01 이후 거래건 삭제
customer_category_df["purchase_time"] = pd.to_datetime(
    customer_category_df["purchase_time"], 
    format="%Y-%m-%d %H:%M:%S"
)
# 115,055 rows x 30 cols
customer_df = customer_category_df[(customer_category_df["purchase_time"] >= "2017-01-01") & (customer_category_df["purchase_time"] < "2018-09-01")]

# 중복열 제거        115,055 -> 114,649
customer_df = customer_df.drop_duplicates()

In [10]:
customer_df

,customer_id,customer_unique_id,zip_code,latitude,longitude,city,state,order_id,order_status,purchase_time,approved_time,carrier_delivered_time,delivered_time,estimated_delivery_time,payment_sequential,payment_type,payment_installments,payment_amount,quantity,product_id,carrier_delivered_limit_time,price,freight_value,review_score,product_category_name,content_length,photos_qty,weight_g,length,height,width,product_category_name_english,category
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,-20.513713,-47.396644,franca,SP,00e7ee1b050b8499577073aeb2a297a1,delivered,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05 00:00:00,1.0,credit_card,2.0,146.87,1,a9516a079e37a9c9c36b9b78b10169e8,2017-05-22 15:22:12,124.99,21.88,4.0,moveis_escritorio,1141.0,1.0,8683.0,54.0,64.0,31.0,office_furniture,가구·인테리어
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,-23.724495,-46.548297,sao bernardo do campo,SP,29150127e6685892b6eab3eec79f59c7,delivered,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06 00:00:00,1.0,credit_card,8.0,335.48,1,4aa6014eceb682077f9dc4bffebc05b0,2018-01-18 20:58:32,289.00,46.48,5.0,utilidades_domesticas,1002.0,3.0,10150.0,89.0,15.0,40.0,housewares,생활용품
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,-23.531294,-46.656404,sao paulo,SP,b2059ed67ce144a36e2aa97d2c9e9ad2,delivered,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13 00:00:00,1.0,credit_card,7.0,157.73,1,bd07b66896d6f1494f5b86251848ced7,2018-06-05 16:19:10,139.94,17.79,5.0,moveis_escritorio,955.0,1.0,8267.0,52.0,52.0,17.0,office_furniture,가구·인테리어
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,-23.496930,-46.185352,mogi das cruzes,SP,951670f92359f4fe4a63112aa7306eba,delivered,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10 00:00:00,1.0,credit_card,1.0,173.30,1,a5647c44af977b148e0a3a4751a09e2e,2018-03-27 16:31:16,149.94,23.36,5.0,moveis_escritorio,1066.0,1.0,12160.0,56.0,51.0,28.0,office_furniture,가구·인테리어
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,-22.972931,-47.140439,campinas,SP,6b7d50bd145f6fc7f33cebabd7e49d0f,delivered,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15 00:00:00,1.0,credit_card,8.0,252.25,1,9391a573abe00141c56e38d84d7d5b3b,2018-07-31 10:10:09,230.00,22.25,5.0,casa_conforto,407.0,1.0,5200.0,45.0,15.0,35.0,home_confort,가구·인테리어
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113062,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,-23.583469,-46.499797,sao paulo,SP,6760e20addcf0121e9d58f2f1ff14298,delivered,2018-04-07 15:48:17,2018-04-07 16:08:45,2018-04-11 02:08:36,2018-04-13 20:06:37,2018-04-25 00:00:00,1.0,credit_card,6.0,88.78,1,ccb4503d9d43d245d3b295d0544f988b,2018-04-12 16:08:45,74.90,13.88,4.0,livros_interesse_geral,489.0,2.0,611.0,22.0,22.0,23.0,books_general_interest,도서·미디어
113063,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,-23.612294,-46.765787,taboao da serra,SP,9ec0c8947d973db4f4e8dcf1fbfa8f1b,delivered,2018-04-04 08:20:22,2018-04-04 08:35:12,2018-04-05 18:42:35,2018-04-11 18:54:45,2018-04-20 00:00:00,1.0,credit_card,3.0,129.06,1,9ede6b0570a75a4b9de4f383329f99ee,2018-04-10 08:35:12,114.90,14.16,5.0,esporte_lazer,1193.0,1.0,1211.0,25.0,24.0,22.0,sports_leisure,스포츠·레저
113064,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,-3.729566,-38.507380,fortaleza,CE,fed4434add09a6f332ea398efd656a5c,delivered,2018-04-08 20:11:50,2018-04-08 20:30:03,2018-04-09 17:52:17,2018-05-09 19:03:15,2018-05-02 00:00:00,1.0,credit_card,5.0,56.04,1,7a5d2e1e131a860ae7d18f6fffa9d689,2018-04-12 20:30:03,37.00,19.04,1.0,beleza_saude,575.0,1.0,870.0,25.0,20.0,18.0,health_beauty,건강·뷰

In [11]:
customer_df.groupby("order_id").agg({
    'quantity': 'nunique',
    'customer_unique_id': 'count'
})

,quantity,customer_unique_id
order_id,,
00010242fe8c5a6d1ba2dd792cb16214,1,1
00018f77f2f0320c557190d7a144bdd3,1,1
000229ec398224ef6ca0657da4fc703e,1,1
00024acbcdf0a6daa1e931b038114c75,1,1
00042b26cf59d7ce69dfabb4e55b4fd9,1,1
...,...,...
fffc94f6ce00a00581880bf54a75a037,1,1
fffcd46ef2263f404302a634eb57f7eb,1,1
fffce4705a9662cd70adb13d4a31832d,1,1


In [12]:
customer_df[customer_df['order_id'] == '0016dfedd97fc2950e388d2971d718c7']

,customer_id,customer_unique_id,zip_code,latitude,longitude,city,state,order_id,order_status,purchase_time,approved_time,carrier_delivered_time,delivered_time,estimated_delivery_time,payment_sequential,payment_type,payment_installments,payment_amount,quantity,product_id,carrier_delivered_limit_time,price,freight_value,review_score,product_category_name,content_length,photos_qty,weight_g,length,height,width,product_category_name_english,category
82829,2c8b917c5d7dd720ebe36a5ed3b501ec,33febb33a42a35f5826a22584b127f9d,45860,-15.669929,-38.952963,canavieiras,BA,0016dfedd97fc2950e388d2971d718c7,delivered,2017-04-28 19:54:40,2017-04-29 10:05:12,2017-05-16 08:40:49,2017-05-22 18:32:21,2017-06-01 00:00:00,2.0,voucher,1.0,17.92,1,4089861a1bd4685da70bddd6b4f974f1,2017-05-05 10:05:12,49.75,20.8,5.0,ferramentas_jardim,152.0,1.0,1500.0,27.0,6.0,23.0,garden_tools,건축·공구
82830,2c8b917c5d7dd720ebe36a5ed3b501ec,33febb33a42a35f5826a22584b127f9d,45860,-15.669929,-38.952963,canavieiras,BA,0016dfedd97fc2950e388d2971d718c7,delivered,2017-04-28 19:54:40,2017-04-29 10:05:12,2017-05-16 08:40:49,2017-05-22 18:32:21,2017-06-01 00:00:00,1.0,credit_card,5.0,52.63,1,4089861a1bd4685da70bddd6b4f974f1,2017-05-05 10:05:12,49.75,20.8,5.0,ferramentas_jardim,152.0,1.0,1500.0,27.0,6.0,23.0,garden_tools,건축·공구


In [13]:
# 9 파생변수 생성 및 컬럼 정리
# order_qty: 주문별 수량
customer_df["product_qty"] = customer_df.groupby(["order_id", "product_id"])["product_id"].transform("count")
customer_df["order_qty"] = customer_df.groupby("order_id")["quantity"].transform("nunique")

# total_price 파생변수 신규 생성
customer_df["total_price"] = customer_df.groupby(["order_id", "customer_id"])["payment_amount"].transform("sum")

# payment_sequential 개수 때문에 중복되는 행들 제거 >>NEW<<
customer_df["qty"] = customer_df.groupby(["order_id", "customer_id", "product_id"])["quantity"].transform("max")
filtered1_df = customer_df[customer_df["quantity"] == customer_df["qty"]]
filtered1_df = filtered1_df.drop(columns=["quantity"])

customer_df["payment_sequence"] = customer_df.groupby(["order_id", "customer_id", "product_id"])["payment_sequential"].transform("max")
filtered2_df = customer_df[customer_df["payment_sequential"] == customer_df["payment_sequence"]]
filtered2_df = filtered2_df.drop(columns=["payment_sequential"])

# 컬럼명 순서 정렬
customer_order = ["order_id", "order_qty", "total_price", "product_id", "product_qty", "category", "review_score", "price", "freight_value", 
                  "purchase_time", "approved_time", "carrier_delivered_time", "delivered_time", "estimated_delivery_time", "payment_type", "payment_installments", 
                  "content_length", "photos_qty", "weight_g", "length", "height", "width",
                  "zip_code", "latitude", "longitude", "city", "state", "customer_id", "customer_unique_id"]
customer_df = filtered2_df[customer_order]

# 중복행 제거
customer_df = customer_df.drop_duplicates()
# # 4개 컬럼 기준 중복 행 제거
# customer_df = customer_df.drop_duplicates(subset=["order_id", "customer_unique_id", "delivered_time", "product_id", "price"])

# # 데이터 타입 변환
customer_df["zip_code"] = customer_df["zip_code"].astype(int)
customer_df["product_qty"] = customer_df["product_qty"].astype(int)
customer_df["order_qty"] = customer_df["order_qty"].astype(int)
# customer_df["payment_sequential"] = customer_df["payment_sequential"].astype(int)
customer_df["payment_installments"] = customer_df["payment_installments"].astype(int)
customer_df["content_length"] = customer_df["content_length"].astype(int)
customer_df["photos_qty"] = customer_df["photos_qty"].astype(int)
# 날짜 관련 컬럼들을 datetime 타입으로 변환
time_convert = ["purchase_time", "approved_time", "carrier_delivered_time", "delivered_time", "estimated_delivery_time", "carrier_delivered_time"]
for i in time_convert:
    customer_df[i] = pd.to_datetime(customer_df[i])

# 03 판매자 데이터프레임 생성

In [14]:
### 1 sellers_df + geo_mode_df              (3095, 7) -> (3088, 6)
# 판매자 우편번호에서 위치정보 데이터프레임에도 존재하지 않는 우편번호 7개 삭제
# 중복되는 필요없는 컬럼 삭제
merged_seller_df = pd.merge(sellers_df, geo_df, left_on="seller_zip_code_prefix", right_on="zip_code", how="left")
merged_seller_df.dropna(subset=["zip_code"], inplace=True)
merged_seller_df.drop(columns=["seller_zip_code_prefix"], inplace=True)

# -> 판매자의 위치정보가 결측치일경우 0으로 대체
### 2 order_items_df + geo_mode_df
# zip_code, latitude, longitide, city, state NULL값이라 0으로 대체
seller_orders_items_df = pd.merge(order_items_df, merged_seller_df, on="seller_id", how="inner")

### 3 + order_reviews_df
seller_reviews_df = pd.merge(seller_orders_items_df, order_reviews_df, on="order_id", how="left")
seller_reviews_df["review_score"] = seller_reviews_df["review_score"].fillna("리뷰없음")

### 4 + products_df
seller_product_df = pd.merge(seller_reviews_df, products_df, on="product_id", how="left")
seller_product_df.dropna(subset=["product_category_name"], inplace=True)

### 5 + category_df
#카테고리가 결측치일경우 카테고리 없음으로 대체 
seller_category_df = pd.merge(seller_product_df, category_df, on="product_category_name", how="left")
seller_category_df["category"].fillna("카테고리없음", inplace=True)


### 6 필터링 및 컬럼 정리                       (113314, 21)  ->  (102093, 16)
# 2017-01-01 이후 거래건 + 2018-09-01 이후 거래건 삭제
seller_df = seller_category_df[(seller_category_df["carrier_delivered_limit_time"] >= "2017-01-01") & (seller_category_df["carrier_delivered_limit_time"] < "2018-09-01")]

### 7 파생변수 생성 및 컬럼 정리
# payment_sequential 개수 때문에 중복되는 행들 제거 >>NEW<<
seller_df["qty"] = seller_df.groupby(["order_id", "seller_id", "product_id"])["quantity"].transform("max")
filtered3_df = seller_df[seller_df["quantity"] == seller_df["qty"]]
filtered3_df = filtered3_df.drop(columns=["quantity"])


# 컬럼 순서 변경
seller_order = ["order_id", "product_id", "category", "review_score", "price", "freight_value", 
                "content_length", "photos_qty", "weight_g", "length", "height", "width",
                "zip_code", "latitude", "longitude", "city", "state", "seller_id"]
seller_df = seller_df[seller_order]


# # 중복행 제거         112,837 -> 102,232
seller_df = seller_df.drop_duplicates()

# 데이터 변환
seller_df["zip_code"] = seller_df["zip_code"].astype(int)
seller_df["content_length"] = seller_df["content_length"].astype(int)
seller_df["photos_qty"] = seller_df["photos_qty"].astype(int)
seller_df["photos_qty"] = seller_df["photos_qty"].astype(int)

In [15]:
customer_df["zip_code"] = customer_df["zip_code"].astype(str)
seller_df["zip_code"] = seller_df["zip_code"].astype(str)

In [16]:
customer_df.to_csv(DATA_DIR / 'processed/customer_data.csv', index=False, date_format='%Y-%m-%d %H:%M:%S')
seller_df.to_csv(DATA_DIR / 'processed/seller_data.csv', index=False, date_format='%Y-%m-%d %H:%M:%S')

In [17]:
processed_df = pd.merge(customer_df, seller_df[['order_id', 'seller_id', 'zip_code', 'city', 'state', 'latitude', 'longitude']], how='inner', on='order_id', suffixes=('_cust', '_sell')).drop_duplicates()
processed_df

,order_id,order_qty,total_price,product_id,product_qty,category,review_score,price,freight_value,purchase_time,approved_time,carrier_delivered_time,delivered_time,estimated_delivery_time,payment_type,payment_installments,content_length,photos_qty,weight_g,length,height,width,zip_code_cust,latitude_cust,longitude_cust,city_cust,state_cust,customer_id,customer_unique_id,seller_id,zip_code_sell,city_sell,state_sell,latitude_sell,longitude_sell
0,00e7ee1b050b8499577073aeb2a297a1,1,146.87,a9516a079e37a9c9c36b9b78b10169e8,1,가구·인테리어,4.0,124.99,21.88,2017-05-16 15:05:35,2017-05-16 15:22:12,2017-05-23 10:47:57,2017-05-25 10:35:35,2017-06-05,credit_card,2,1141,1,8683.0,54.0,64.0,31.0,14409,-20.513713,-47.396644,franca,SP,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,7c67e1448b00f6e969d365cea6b010ab,8577,itaquaquecetuba,SP,-23.479473,-46.360819
1,29150127e6685892b6eab3eec79f59c7,1,335.48,4aa6014eceb682077f9dc4bffebc05b0,1,생활용품,5.0,289.00,46.48,2018-01-12 20:48:24,2018-01-12 20:58:32,2018-01-15 17:14:59,2018-01-29 12:41:19,2018-02-06,credit_card,8,1002,3,10150.0,89.0,15.0,40.0,9790,-23.724495,-46.548297,sao bernardo do campo,SP,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,b8bc237ba3788b23da09c0f1f3a3288c,88303,itajai,SC,-26.920912,-48.684937
2,b2059ed67ce144a36e2aa97d2c9e9ad2,1,157.73,bd07b66896d6f1494f5b86251848ced7,1,가구·인테리어,5.0,139.94,17.79,2018-05-19 16:07:45,2018-05-20 16:19:10,2018-06-11 14:31:00,2018-06-14 17:58:51,2018-06-13,credit_card,7,955,1,8267.0,52.0,52.0,17.0,1151,-23.531294,-46.656404,sao paulo,SP,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,7c67e1448b00f6e969d365cea6b010ab,8577,itaquaquecetuba,SP,-23.479473,-46.360819
3,951670f92359f4fe4a63112aa7306eba,1,173.30,a5647c44af977b148e0a3a4751a09e2e,1,가구·인테리어,5.0,149.94,23.36,2018-03-13 16:06:38,2018-03-13 17:29:19,2018-03-27 23:22:42,2018-03-28 16:04:25,2018-04-10,credit_card,1,1066,1,12160.0,56.0,51.0,28.0,8775,-23.496930,-46.185352,mogi das cruzes,SP,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,7c67e1448b00f6e969d365cea6b010ab,8577,itaquaquecetuba,SP,-23.479473,-46.360819
4,6b7d50bd145f6fc7f33cebabd7e49d0f,1,252.25,9391a573abe00141c56e38d84d7d5b3b,1,가구·인테리어,5.0,230.00,22.25,2018-07-29 09:51:30,2018-07-29 10:10:09,2018-07-30 15:16:00,2018-08-09 20:55:48,2018-08-15,credit_card,8,407,1,5200.0,45.0,15.0,35.0,13056,-22.972931,-47.140439,campinas,SP,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,4a3ca9315b744ce9f8e9374361493884,14940,ibitinga,SP,-21.774868,-48.847152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106436,6760e20addcf0121e9d58f2f1ff14298,1,88.78,ccb4503d9d43d245d3b295d0544f988b,1,도서·미디어,4.0,74.90,13.88,2018-04-07 15:48:17,2018-04-07 16:08:45,2018-04-11 02:08:36,2018-04-13 20:06:37,2018-04-25,credit_card,6,489,2,611.0,22.0,22.0,23.0,3937,-23.583469,-46.499797,sao paulo,SP,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,527801b552d0077ffd170872eb49683b,17400,garca,SP,-22.210402,-49.662731
106437,9ec0c8947d973db4f4e8dcf1fbfa8f1b,1,129.06,9ede6b0570a75a4b9de4f383329f99ee,1,스포츠·레저,5.0,114.90,14.16,2018-04-04 08:20:22,2018-04-04 08:35:12,2018-04-05 18:42:35,2018-04-11 18:54:45,2018-04-20,credit_card,3,1193,1,1211.0,25.0,24.0,22.0,6764,-23.612294,-46.765787,taboao da serra,SP,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,3fd1e727ba94cfe122d165e176ce7967,14802,araraquara,SP,-21.775406,-48.168105
106438,fed4434add09a6f332ea398efd656a5c,1,56.04,7a5d2e1e131a860ae7d18f6fffa9d689,1,건강·뷰티,1.0,37.00,19.04,2018-04-08 20:11:50,2018-04-08 20:30:03,2018-04-09 17:52:17,2018-05-09 19:03:15,2018-05-02,credit_card,5,575,1,870.0,25.0,20.0,18.0,60115,-3.729566,-38.507380,fortaleza,CE,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,d9e7e7778b32987280a6f2cb9a39c57d,3304,sao paulo,SP,-23.545397,-46.581221
106439,e31ec91cea1ecf97797787471f98a8c

In [18]:
processed_df.to_csv('../data/processed/processed_data.csv', index=False, date_format='%Y-%m-%d %H:%M:%S')